#**PONTIFÍCIA UNIVERSIDADE CATÓLICA DE MINAS GERAIS**
##**PÓS-GRADUAÇÃO LATO SENSU EM CIÊNCIA DE DADOS E BIG DATA**
###***TCC - TRABALHO DE CONCLUSÃO DE CURSO:***
###***APLICAÇÃO DE MODELOS ESTATÍSTICOS E DE APRENDIZADO DE MÁQUINA PARA A ESTIMATIVA DO PREÇO DA GASOLINA EM FUNÇÃO DE VARIÁVEIS ECONÔMICAS***
# Discente: Henrique Viegas Cunha
<hr>

#***PROJETO FUEL_PRICE: extração de dados e tratamento inicial***
### O presente *notebook* visa a obtenção dos dados (séries temporais) das diversas variáveis econômicas bem como seu tratamento inicial para a formação do dataframe de trabalho
<hr> 


### *Montando* o drive do *google* para a leitura de arquivos no Google Drive e a exportação do dataframe de trabalho
<hr WIDTH=30% ALIGN=LEFT NOSHADE>

In [ ]:
# "MONTANDO" o drive do Google Drive
# Load the Drive helper and mount (carregando)
from google.colab import drive
# This will prompt for authorization (prompt de autorização)
drive.mount('/content/drive')#, force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importando as bibliotecas que serão utilizadas
<hr WIDTH=30% ALIGN=LEFT NOSHADE>

In [ ]:
# Importando as bibliotecas que serão utilizadas:
import pandas as pd
import numpy as np

# Biblioteca para a "leitura" dos arquivos:
import os
#from os import listdir
#from os.path import isfile, join

# Biblioteca para tratamento de erros:
import warnings

# Configurações da biblioteca de tratamento de erros:
warnings.filterwarnings('ignore')

# Instalando e importando a biblioteca yfinance para extração de dados da bolsa de valores:
!pip install yfinance
import yfinance as yf

# Biblioteca para leitura e manipulação de documentos em excel
import xlrd
!pip install --upgrade xlrd
#import openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Definindo parâmetros de apresentação dos dados:
pd.set_option('display.max_columns', 30) #None) 
pd.set_option('display.max_rows', 30) #None) 
pd.set_option('display.max_colwidth', 30) #None)
#Definindo parâmetros de apresentação dos dados:
#pd.reset_option('display.max_columns') #None) 
#pd.reset_option('display.max_rows') #None) 
#pd.reset_option('display.max_colwidth') #None)

# **Extração dos dados**
### **Capítulo 2**
<hr>

# **Preço combustível/gasolina**
### **Seção 2.1**
### Preço aferido dos combustíveis nos postos brasileiros
<hr>

Preços dos combustíveis: dados abertos que foram extraídos da Agência Nacional do Petróleo, Gás natural e Biocombustíveis no portal *gov.br*:

- **Link**: https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/serie-historica-de-precos-de-combustiveis

Decidiu-se por utilizar os preços disponíveis para as **diversas capitais do país**, pois assim é garantido o uso de um número significativo de dadosmantendo a representatividade regional e populacional, sem um exagerado sacrifício computacional 
<hr WIDTH=30% ALIGN=LEFT NOSHADE>

In [ ]:
# Caminho do diretório onde se encontram os arquivos com os valores de combustível:
caminho = r'/content/drive/MyDrive/NOVO_TCC/DADOS/COMBUSTIVEL/'
#caminho = r'C:\Users\Henrique\Documents\TCC-PUCMG\DADOS\COMBUSTIVEL\'


# Listando todos os arquivos contantes do diretório de caminho: 
arqs_combustivel = sorted([
    f"{caminho}{arq}" for arq in os.listdir(caminho) if os.path.isfile(os.path.join(caminho, arq))
])

# Aprensentando a lista com o nome dos arquivos:
print ('Lista de arquivos com os dados de preço de combustíveis: \n')
arqs_combustivel[:]

Lista de arquivos com os dados de preço de combustíveis: 



['/content/drive/MyDrive/NOVO_TCC/DADOS/COMBUSTIVEL/ca-2011-01.csv',
 '/content/drive/MyDrive/NOVO_TCC/DADOS/COMBUSTIVEL/ca-2011-02.csv',
 '/content/drive/MyDrive/NOVO_TCC/DADOS/COMBUSTIVEL/ca-2012-01.csv',
 '/content/drive/MyDrive/NOVO_TCC/DADOS/COMBUSTIVEL/ca-2012-02.csv',
 '/content/drive/MyDrive/NOVO_TCC/DADOS/COMBUSTIVEL/ca-2013-01.csv',
 '/content/drive/MyDrive/NOVO_TCC/DADOS/COMBUSTIVEL/ca-2013-02.csv',
 '/content/drive/MyDrive/NOVO_TCC/DADOS/COMBUSTIVEL/ca-2014-01.csv',
 '/content/drive/MyDrive/NOVO_TCC/DADOS/COMBUSTIVEL/ca-2014-02.csv',
 '/content/drive/MyDrive/NOVO_TCC/DADOS/COMBUSTIVEL/ca-2015-01.csv',
 '/content/drive/MyDrive/NOVO_TCC/DADOS/COMBUSTIVEL/ca-2015-02.csv',
 '/content/drive/MyDrive/NOVO_TCC/DADOS/COMBUSTIVEL/ca-2016-01.csv',
 '/content/drive/MyDrive/NOVO_TCC/DADOS/COMBUSTIVEL/ca-2016-02.csv',
 '/content/drive/MyDrive/NOVO_TCC/DADOS/COMBUSTIVEL/ca-2017-01.csv',
 '/content/drive/MyDrive/NOVO_TCC/DADOS/COMBUSTIVEL/ca-2017-02.csv',
 '/content/drive/MyDrive/NOVO_TCC/

### A seguir temos as funções para tratar inicialmente os dados e disponibilizá-los com o melhor formato possível para as próximas etapas
### Além disso, vamos extrair a data e o valor médio de venda de gasolina para aquela data
<hr WIDTH=30% ALIGN=LEFT NOSHADE>

In [ ]:
# Função para extrair e filtrar os dados
# unindo todos os arquivos csv
def extrai_valor_gas(df_list):
    
    # Captura os dados nos arquvos csv e filtra apenas o que nos interessa:
    # cidade: 'SAO PAULO' / produto:'GASOLINA'
    # Data da coleta e valor de venda
    
    CIDADES = ['SAO PAULO', 'RIO DE JANEIRO', 'BRASILIA', 'SALVADOR', 'FORTALEZA', 'BELO HORIZONTE',
               'MANAUS', 'CURITIBA', 'RECIFE', 'GOIANIA', 'RIO BRANCO', 'MACEIO', 'MACAPA', 'VITORIA',
               'SAO LUIS', 'CUIABA', 'CAMPO GRANDE', 'BELEM', 'JOAO PESSOA', 'TERESINA', 'NATAL',
               'PORTO ALEGRE', 'PORTO VELHO', 'BOA VISTA', 'FLORIANOPOIS','ARACAJU', 'PALMAS']
    PRODUTO = 'GASOLINA'
    COLS = ['Data da Coleta', 'Valor de Venda']
    #COLS = ['Data da Coleta', 'Valor de Venda', 'Municipio']

    df_final = pd.DataFrame()

    for df_path in df_list:
        tmp = pd.read_csv(df_path, sep=';')
        tmp = tmp[
            (tmp['Municipio'].isin(CIDADES)) & 
            (tmp['Produto'] == PRODUTO)
        ]
        df_final = df_final.append(tmp[COLS])

    return df_final

# Função para tratar o formato dos dados numéricos, 
# Substituindo a notação brasileira pela americana
def ajusta_valor(df, column):
      
    #Transforma os dados de str para float
    df[column] = df[column].str.replace(',', '.')
    #Removendo a vírgula e substituindo por ponto
    df[column] = df[column].astype(float)

    return df

# Função para converter os formatos de data
def datetime_format(df, column, date_format='%d/%m/%Y'):
    
    # Transformando as colunas de data em seu formato padrão
    df[column] = pd.to_datetime(
        df[column],
        format=date_format
    )

    return df

# Função para a criação de dataframe com os dados ajustados
def cria_df_gas(df):

    # Criando o dataframe por meio das funções pré-definidas:
    # Ajusta valor de venda:
    df_final = ajusta_valor(df, 'Valor de Venda')

    # Ajusta formato data da coleta:
    df_final = datetime_format(df_final, 'Data da Coleta')
    
    # Agrupando os valores de venda pela data e retornando sua média:
    df_final = df_final.groupby(['Data da Coleta']).mean()
    
    return df_final.reset_index()

In [ ]:
# Gerando o dataframe com valor da gasolina
# já aplicadas as transformações:
df_fuel_tmp = extrai_valor_gas(arqs_combustivel)
df_fuel = cria_df_gas(df_fuel_tmp)
df_fuel.head()

,Data da Coleta,Valor de Venda
0,2011-01-03,2.584085
1,2011-01-04,2.637004
2,2011-01-05,2.606665
3,2011-01-06,2.669766
4,2011-01-10,2.614946


In [ ]:
# Gerando o dataframe com valor da gasolina
# já aplicadas as transformações:
df_fuel_tmp = extrai_valor_gas(arqs_combustivel)
df_fuel = cria_df_gas(df_fuel_tmp)

# Renomeando as colunas
df_fuel.columns = ['Date', 'Fuel_Price']

# Resultado
df_fuel.head()

,Date,Fuel_Price
0,2011-01-03,2.584085
1,2011-01-04,2.637004
2,2011-01-05,2.606665
3,2011-01-06,2.669766
4,2011-01-10,2.614946


In [ ]:
df_fuel

,Date,Fuel_Price
0,2011-01-03,2.584085
1,2011-01-04,2.637004
2,2011-01-05,2.606665
3,2011-01-06,2.669766
4,2011-01-10,2.614946
...,...,...
2297,2022-01-24,6.672293
2298,2022-01-25,6.658731
2299,2022-01-26,6.637217
2300,2022-01-27,6.680007


In [ ]:
# Resultado
print(f'Foram obtidas {df_fuel.shape[0]} linhas de dados com preços da gasolina.')

Foram obtidas 2302 linhas de dados com preços da gasolina.


In [ ]:
# Forma do dataset com o preço da gasolina
df_fuel.shape

(2302, 2)

In [ ]:
# BackUp
#arq_exportado = r'/content/drive/MyDrive/NOVO_TCC/DADOS/BACKUP_DF/df_fuel.csv'
arq_exportado = r'C:\Users\Henrique\Documents\TCC-PUCMG\DADOS\BACKUP_DF\df_fuel.csv'
df_fuel.to_csv(arq_exportado)

# **Ações Petrobras**
### **Seção 2.2**
### Extraindo os valores de cotação das ações Petrobras. 

* **Código da ação**: PETR4

### Dados constantes do dataframe gerado pelo *yfinance* (obs.: todos os valores são em dólares americanos):

*   Data do pregão da bolsa de valores
*   Valor da ação na abertura do pregão
*   Maior valor da ação no dia 
*   Menor valor da ação no dia
*   Valor da ação no fechamento do pregão
*   Valor da ação ajustado no fechamento do pregão
*   O volume comercializado


 
<hr WIDTH=30% ALIGN=LEFT NOSHADE>

In [ ]:
# Estraindo os dados da ação da Petrobras, PETR4, de 01/01/2011 a 31/01/2022:
df_petr4 = yf.download("PETR4.SA", start="2011-01-01", end="2022-01-31")
df_petr4.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2011-01-03,27.500000,27.580000,26.91,27.000000,17.614801,22738500
2011-01-04,27.010000,27.059999,26.40,26.900000,17.549561,30936600
2011-01-05,26.680000,27.350000,26.58,27.219999,17.758326,24306800
2011-01-06,27.200001,27.459999,27.01,27.110001,17.686563,24328800
2011-01-07,27.020000,27.180000,26.73,26.730000,17.438652,17453800


In [ ]:
# Criando uma coluna com as datas de coleta das informações da ação:
df_petr4.insert(0, 'Date', df_petr4.index)
# Capturando apenas as colunas de interesse (data e valor de fechamento da ação)
df_petr4 = df_petr4[['Date', 'Close']]
df_petr4.head()

,Date,Close
Date,,
2011-01-03,2011-01-03,27.000000
2011-01-04,2011-01-04,26.900000
2011-01-05,2011-01-05,27.219999
2011-01-06,2011-01-06,27.110001
2011-01-07,2011-01-07,26.730000


In [ ]:
# Renomenado as colunas
df_petr4.columns = ['Date', 'PETR4']
# Resetando o index:
df_petr4.reset_index(drop=True, inplace=True)
df_petr4.head()

,Date,PETR4
0,2011-01-03,27.000000
1,2011-01-04,26.900000
2,2011-01-05,27.219999
3,2011-01-06,27.110001
4,2011-01-07,26.730000


In [ ]:
df_petr4

,Date,PETR4
0,2011-01-03,27.000000
1,2011-01-04,26.900000
2,2011-01-05,27.219999
3,2011-01-06,27.110001
4,2011-01-07,26.730000
...,...,...
2743,2022-01-24,31.950001
2744,2022-01-25,32.990002
2745,2022-01-26,33.869999
2746,2022-01-27,33.880001


In [ ]:
# Shape do dataframe:
df_petr4.shape

(2748, 2)

In [ ]:
# Resultado
print(f'Foram obtidas {df_petr4.shape[0]} linhas de dados com valores das ações PETR4.')

Foram obtidas 2748 linhas de dados com valores das ações PETR4.


In [ ]:
# BackUp
# arq_exportado = r'/content/drive/MyDrive/NOVO_TCC/DADOS/BACKUP_DF/df_petr4.csv'
arq_exportado = r'C:\Users\Henrique\Documents\TCC-PUCMG\DADOS\BACKUP_DF\df_petr4.csv'
df_petr4.to_csv(arq_exportado)

# **Cotação do dólar**
### **Seção 2.3**
### Extraindo os valores de cotações diárias do valor do dólar americano em relação ao real (preço do dólar em real)
### Os dados foram capturados no portal *Investing.com*

- **Link**: https://br.investing.com/currencies/usd-brl-historical-data


<hr WIDTH=30% ALIGN=LEFT NOSHADE>

In [ ]:
# Função para ajustar os dados obtidos
# no portal investing (preco do dólar)
def ajusta_investing(df):
    # Colunas que serão filtradas do arquivo csv:
    COLS = ['Data', 'Último']
    
    # Ajusta o tipo de dados da coluna de data
    # usando a funcao datetime_format:
    df = datetime_format(df[COLS], 'Data', '%d.%m.%Y')
    # Ajusta o tipo de dados da coluna de preço do dólar
    # de fechamento, usando a funcao ajusta valor:
    df = ajusta_valor(df, 'Último')
    # Ordena os valores por data 
    df.sort_values('Data', inplace=True)

    # Retorna o dataframe com o index resetado:
    return df.reset_index(drop=True)

In [ ]:
# Importação dos dados do preço do dólar:
# arq_importado = r'/content/drive/MyDrive/NOVO_TCC/DADOS/DOLAR/USD_BRL Dados Históricos.csv'
arq_importado = r'C:\Users\Henrique\Documents\TCC-PUCMG\DADOS\DOLAR\USD_BRL Dados Históricos.csv'
df_usd_temp = pd.read_csv(arq_importado)

# Aplicando as transformações (seleção de colunas e alteração dos tipos)
df_usd = ajusta_investing(df_usd_temp)

# Renomeando as colunas
df_usd.columns =['Date', 'USD/BRL']
df_usd.head()

,Date,USD/BRL
0,2011-01-03,1.6470
1,2011-01-04,1.6610
2,2011-01-05,1.6737
3,2011-01-06,1.6856
4,2011-01-07,1.6832


In [ ]:
# Forma do dataframe df_usd
df_usd.shape

(2891, 2)

In [ ]:
# Resultado
print(f'Foram obtidas {df_usd.shape[0]} linhas de dados com cotações do dólar x real.')

Foram obtidas 2891 linhas de dados com cotações do dólar x real.


In [ ]:
df_usd

,Date,USD/BRL
0,2011-01-03,1.6470
1,2011-01-04,1.6610
2,2011-01-05,1.6737
3,2011-01-06,1.6856
4,2011-01-07,1.6832
...,...,...
2886,2022-01-25,5.4442
2887,2022-01-26,5.4329
2888,2022-01-27,5.4070
2889,2022-01-28,5.3676


In [ ]:
# BackUp
# arq_exportado = r'/content/drive/MyDrive/NOVO_TCC/DADOS/BACKUP_DF/df_usd.csv'
arq_exportado = r'C:\Users\Henrique\Documents\TCC-PUCMG\DADOS\BACKUP_DF\df_usd.csv'
df_usd.to_csv(arq_exportado)

# **IPCA**
### **Seção 2.4**
### Obtendo os dados referentes ao Índice Nacional de Preços ao Consumidor-Amplo (IPCA) .
### Os dados foram extraídos do portal *dados.gov.br*

- **Link**: https://dados.gov.br/dataset/4447-indice-nacional-de-precos-ao-consumidor-amplo-ipca-comercializaveis

<hr WIDTH=30% ALIGN=LEFT NOSHADE>

In [ ]:
# Função para ajustar os dados do IPCA
def ajusta_ipca(df):
  
    # Para converter a coluna de data
    # no formato datetime:
    df = datetime_format(df, 'data', '%d/%m/%Y')
    # Ajustando o valor do índice 
    # para o formato float:
    df = ajusta_valor(df, 'valor')
    
    # Limitando a data de interesse:
    df = df[df.data >= '2011-01-01']

    return df

In [ ]:
# Importando os dados do IPCA:
arq_importado = r'/content/drive/MyDrive/NOVO_TCC/DADOS/IPCA/bcdata.sgs.4447.csv'
#arq_importado = r'C:\Users\Henrique\Documents\TCC-PUCMG\DADOS\IPCA\bcdata.sgs.4447.csv'
df_ipca_tmp = pd.read_csv(arq_importado, sep=';')

# Aplicando os ajustes ao IPCA:
df_ipca = ajusta_ipca(df_ipca_tmp)

# Renomeando as colunas e resetando o index:
df_ipca.columns = ['Date', 'IPCA']
df_ipca.reset_index(drop=True, inplace=True)
df_ipca.head()

,Date,IPCA
0,2011-01-01,0.43
1,2011-02-01,-0.02
2,2011-03-01,0.27
3,2011-04-01,0.60
4,2011-05-01,0.36


In [ ]:
# Forma do dataframe com dados do IPCA
df_ipca.shape

(134, 2)

In [ ]:
# Resultado
print(f'Foram obtidas {df_ipca.shape[0]} linhas de dados com os índices do IPCA.')

Foram obtidas 134 linhas de dados com os índices do IPCA.


In [ ]:
df_ipca

,Date,IPCA
0,2011-01-01,0.43
1,2011-02-01,-0.02
2,2011-03-01,0.27
3,2011-04-01,0.60
4,2011-05-01,0.36
...,...,...
129,2021-10-01,1.22
130,2021-11-01,0.58
131,2021-12-01,1.58
132,2022-01-01,1.15


In [ ]:
# BackUp
#arq_exportado = r'/content/drive/MyDrive/NOVO_TCC/DADOS/BACKUP_DF/df_ipca.csv'
arq_exportado = r'C:\Users\Henrique\Documents\TCC-PUCMG\DADOS\BACKUP_DF\df_ipca.csv'
df_ipca.to_csv(arq_exportado)

# **Petróleo Brent**
### **Seção 2.5**
### Obtendo os dados referentes à cotação internacional do preço do barril de petróleo tipo Brent
### Os dados foram coletados na plataforma da *Investing.com*

- **Link**: https://br.investing.com/commodities/brent-oil-historical-data

<hr WIDTH=30% ALIGN=LEFT NOSHADE>

In [ ]:
# Importando os dados dos barris de petróleo Brent
# arq_importado = r'/content/drive/MyDrive/NOVO_TCC/DADOS/BRENT/Petróleo Brent Futuros Dados Históricos.csv'
arq_importado = r'C:\Users\Henrique\Documents\TCC-PUCMG\DADOS\BRENT\Petróleo Brent Futuros Dados Históricos.csv'
df_brent_tmp = pd.read_csv(arq_importado, sep=',')

# Ajustando os dados
# Nesse caso, não foi necessário criar uma nova função de ajuste
# pois a função utilizada para ajustar o dataframe com a cotação 
# do preço do dólar pode ser usada indistintamente
df_brent = ajusta_investing(df_brent_tmp)

# Renomeando as colunas
df_brent.columns = ['Date', 'Brent']
df_brent.head()

,Date,Brent
0,2011-01-03,94.84
1,2011-01-04,93.53
2,2011-01-05,95.50
3,2011-01-06,94.52
4,2011-01-07,93.33


In [ ]:
# Formato do dataframe Brent:
df_brent.shape

(2862, 2)

In [ ]:
# Resultado
print(f'Foram obtidas {df_brent.shape[0]} linhas de dados'
      f'com valores de cotação do barril de petróleo Brent.')

Foram obtidas 2862 linhas de dadoscom valores de cotação do barril de petróleo Brent.


In [ ]:
df_brent

,Date,Brent
0,2011-01-03,94.84
1,2011-01-04,93.53
2,2011-01-05,95.50
3,2011-01-06,94.52
4,2011-01-07,93.33
...,...,...
2857,2022-01-25,88.20
2858,2022-01-26,88.74
2859,2022-01-27,88.17
2860,2022-01-28,90.03


In [ ]:
# BackUp
# arq_exportado = r'/content/drive/MyDrive/NOVO_TCC/DADOS/BACKUP_DF/df_brent.csv'
arq_exportado = r'C:\Users\Henrique\Documents\TCC-PUCMG\DADOS\BACKUP_DF\df_brent.csv'
df_brent.to_csv(arq_exportado)

# **Consumo de Energia**
### **Seção 2.6**
### Obtendo os dados referentes ao consumo de energia no país
### Os dados referentes ao consumo de energia no Brasil foram retirados da plataforma da *Empresa de Pesquisa Energética (EPE)*

- **Link**: https://www.epe.gov.br/pt/publicacoes-dados-abertos/publicacoes/consumo-de-energia-eletrica 

<hr WIDTH=30% ALIGN=LEFT NOSHADE>

In [ ]:
# Carregando dados energia 
# Considerada apenas a primeira planilhas, a leitura descarta as 3 primeiras linhas da planhilha (cabeçalho)
# e recebe apenas as 13 primeiras colunas, pois aprimeira coluna possui labels de interesse e as demais colunas
# contém labels de ano e/ou mês ou valores do consumo de energia
# arq_importado = r'/content/drive/MyDrive/NOVO_TCC/DADOS/ENERGIA/CONSUMO MENSAL DE ENERGIA ELÉTRICA POR CLASSE.xls'
arq_importado = r'C:\Users\Henrique\Documents\TCC-PUCMG\DADOS\ENERGIA\CONSUMO MENSAL DE ENERGIA ELÉTRICA POR CLASSE.xls'
cols = [0,1,2,3,4,5,6,7,8,9,10,11,12]
df_mwh_tmp = pd.read_excel(arq_importado, skiprows=3, sheet_name=0, usecols=cols)
#df_mwh_tmp[0:19]
#df_mwh_tmp[32:51]
df_mwh_tmp.head()


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,2022*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,JAN,FEV,MAR,ABR,MAI,JUN,JUL,AGO,SET,OUT,NOV,DEZ
2,TOTAL BRASIL,42486943.67,0,0,0,0,0,0,0,0,0,0,0
3,REGIÃO GEOGRÁFICA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Norte,3027551.36,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Selecionando apenas as linhas com valores de interesse
df_mwh_tmp = df_mwh_tmp.loc[((df_mwh_tmp['Unnamed: 0']=='TOTAL BRASIL') |
                             (df_mwh_tmp['Unnamed: 0'].isna())) & 
                             (df_mwh_tmp['Unnamed: 1'].notna())]
# Ajustanto especificamente o anos de 2022, 
# em razão de presença de caracter especial
df_mwh_tmp.loc[0,'Unnamed: 1'] = int(2022)
# Reinicializando o index
df_mwh_tmp.reset_index(drop = True, inplace = True)
df_mwh_tmp.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,JAN,FEV,MAR,ABR,MAI,JUN,JUL,AGO,SET,OUT,NOV,DEZ
2,TOTAL BRASIL,42486943.67,0,0,0,0,0,0,0,0,0,0,0
3,NaN,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,JAN,FEV,MAR,ABR,MAI,JUN,JUL,AGO,SET,OUT,NOV,DEZ


In [ ]:
# Função para ajustar a coluna com o rólulo dos dados:
def ajusta_serie_energia (seriesx):
  list_temp = []
  # A coluna "Unnamed: 0" precisa ter seus textos ajustados
  # pois serão utilizados como rótulos das colunas no dataframe
  # de energia. 
  for indice, valor in seriesx.iteritems():
    # A coluna "Unnamed: 0" tem a sequencia 
    # "ano", "mes" e "total mensal de energia consumida". Ajustando:
    if pd.isna(valor) and indice % 3 == 0:
      list_temp.append('ano')
    elif pd.isna(valor) and indice % 3 != 0:
      list_temp.append('mes')
    else:
      list_temp.append('tot_mwh_mes')
  seriesx = pd.Series(list_temp)
  seriesx.reset_index(drop=True, inplace=True)
  return seriesx

In [ ]:
# Ajsutando a coluna "Unnamed: 0" com os labels de interesse:
df_mwh_tmp.loc[:,'Unnamed: 0'] = ajusta_serie_energia (df_mwh_tmp['Unnamed: 0'])
df_mwh_tmp.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,ano,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mes,JAN,FEV,MAR,ABR,MAI,JUN,JUL,AGO,SET,OUT,NOV,DEZ
2,tot_mwh_mes,42486943.67,0,0,0,0,0,0,0,0,0,0,0
3,ano,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mes,JAN,FEV,MAR,ABR,MAI,JUN,JUL,AGO,SET,OUT,NOV,DEZ


In [ ]:
# Preenchendo os valores nulos dos anos (repetindo o último ano válido):
df_mwh_tmp.fillna(method='ffill', axis=1, inplace=True)

# Transpondo o dataframe para podermos usar os dados e os rótulos gerados:
df_mwh_tmp=df_mwh_tmp.T

# Renomeando as colunas do dataframe com os valores de "Unnamed: 0"
df_mwh_tmp.columns = df_mwh_tmp.loc['Unnamed: 0',:].tolist()
# Deve-se descartar a linha "Unnamed: 0"
df_mwh_tmp.drop(labels='Unnamed: 0', axis=0, inplace=True)
df_mwh_tmp.head()

,ano,mes,tot_mwh_mes,ano,mes,tot_mwh_mes,ano,mes,tot_mwh_mes,ano,...,tot_mwh_mes,ano,mes,tot_mwh_mes,ano,mes,tot_mwh_mes,ano,mes,tot_mwh_mes
Unnamed: 1,2022,JAN,42486943.67,2021,JAN,42530597.78,2020,JAN,41254099.1101,2019,...,30699704,2006,JAN,28880910.62,2005,JAN,28375437.441,2004,JAN,26978358.02
Unnamed: 2,2022,FEV,0,2021,FEV,41254189.529,2020,FEV,40887373.84,2019,...,30324669,2006,FEV,29383486.5,2005,FEV,27838480.97,2004,FEV,26508170.77
Unnamed: 3,2022,MAR,0,2021,MAR,43416334.45,2020,MAR,41061524.489,2019,...,31312900,2006,MAR,29897827.73,2005,MAR,28598631.509,2004,MAR,27284329.41
Unnamed: 4,2022,ABR,0,2021,ABR,42432431.88,2020,ABR,37183601.75,2019,...,32030744,2006,ABR,29887953.01,2005,ABR,29355501.91,2004,ABR,28046182.351
Unnamed: 5,2022,MAI,0,2021,MAI,40350555.83,2020,MAI,36053401.9601,2019,...,31382917,2006,MAI,28925038.56,2005,MAI,28482150.61,2004,MAI,27244506.48


In [ ]:
# Função para criar o dataframe 
# com o consumo de energia
def cria_df_mwh(dfx):
  # Será necessário juntar ao dataframe de energia um conjunto de 3 colunas
  # do dataframe gerado até então: com as informações de "ano", "mes" e "total mensal de energia consumida" 
  num_infos = len(set(dfx.columns))
  df = pd.DataFrame() # Dataframe em branco
  # Indo por tota a extensão do dataframe temporário
  for i in range (int(dfx.shape[1]/num_infos)):
    indice = int(i*num_infos)
    df_temp = dfx.iloc[:,indice:indice+int(num_infos)]
    df = df.append(df_temp)
    
  # Excluindo aqueles meses em que ainda não ná registro do consumo
  df = df.loc[df['tot_mwh_mes']!=0]
  df.reset_index(drop=True, inplace=True) # Reset do index

  # Dicionario para ajustar os rótulos dos meses:
  dict_mes = {'JAN': '01', 'FEV':'02', 'MAR':'03', 'ABR':'04', 'MAI':'05', 'JUN':'06',
              'JUL':'07', 'AGO':'08', 'SET':'09', 'OUT':'10', 'NOV':'11', 'DEZ':'12' }
  # Substituindo os meses:
  df.replace({'mes': dict_mes}, inplace=True)

  # Criando o dataframe final:
  df_new = pd.DataFrame(columns=['Date', 'MWh'])
  # Criando a coluna de data (ainda como string)
  df_new['Date'] = '01'+'-'+df['mes']+'-'+df['ano'].astype(str)

  # Ajustanto o formato da coluna de data, isto é, convertendo o tipo
  # de string para datetime (usando a função prévia datetime_format)
  df_new = datetime_format(df_new, 'Date', '%d-%m-%Y')

  # Criando a coluna de consumo
  df_new['MWh'] = df['tot_mwh_mes']
  df_new['MWh'].apply(lambda x: round(x))
  df_new['MWh'] = df_new['MWh'].astype(int)
  
  return df_new

In [ ]:
# Criando o dataframe de consumo de energia
df_mwh = cria_df_mwh(df_mwh_tmp)
# Limitando o período de análise para as datas de
# medida iguais ou superiores a 01/01/2015:
df_mwh = df_mwh.loc[df_mwh['Date']>='2011-01-01']


In [ ]:
# Shape do dataframe de energia:
df_mwh.shape

(133, 2)

In [ ]:
# Resultado
print(f'Foram obtidas {df_mwh.shape[0]} linhas de dados com valores'
      f'de medição do consumo nacional de energia elétrica.')

Foram obtidas 133 linhas de dados com valoresde medição do consumo nacional de energia elétrica.


In [ ]:
# BackUp
# arq_exportado = r'/content/drive/MyDrive/NOVO_TCC/DADOS/BACKUP_DF/df_mwh.csv'
arq_exportado = r'C:\Users\Henrique\Documents\TCC-PUCMG\DADOS\BACKUP_DF\df_mwh.csv'
df_mwh.to_csv(arq_exportado)

# **Junção dos datasets**
### **Seção 2.7**
### Fazendo a junção dos diversos datasets de variáveis econômicas em um único dataframe

<hr WIDTH=30% ALIGN=LEFT NOSHADE>

In [ ]:
# Criando uma lista com todos os datasets
ds_list = [df_fuel, df_mwh, df_ipca, df_usd, df_petr4]

# Dataframe final será criado por meio da função merge
# O dataframe inicial é df_brent, por ter maior número de registros
df_final = df_brent

# Unindo os datasets pela função merge 
# e a lista de dataframes ds_list
for df in ds_list:
    df_final = pd.merge(df, df_final, on='Date', how='outer')

# Reordenando o dataframe final pela data:
df_final.sort_values(by='Date', inplace=True)
# Resetando o index:
df_final = df_final.reset_index(drop=True)

# Resultado
df_final.head()

,Date,PETR4,USD/BRL,IPCA,MWh,Fuel_Price,Brent
0,2011-01-01,NaN,NaN,0.43,35873744.0,NaN,NaN
1,2011-01-03,27.000000,1.6470,NaN,NaN,2.584085,94.84
2,2011-01-04,26.900000,1.6610,NaN,NaN,2.637004,93.53
3,2011-01-05,27.219999,1.6737,NaN,NaN,2.606665,95.50
4,2011-01-06,27.110001,1.6856,NaN,NaN,2.669766,94.52


In [ ]:
# Shape do dataframe final
df_final.shape

(2970, 7)

In [ ]:
# Resultado
print(f'Foram obtidas {df_final.shape[0]} linhas de dados no dataframe final (df_final).')

Foram obtidas 2970 linhas de dados no dataframe final (df_final).


In [ ]:
df_final

,Date,PETR4,USD/BRL,IPCA,MWh,Fuel_Price,Brent
0,2011-01-01,NaN,NaN,0.43,35873744.0,NaN,NaN
1,2011-01-03,27.000000,1.6470,NaN,NaN,2.584085,94.84
2,2011-01-04,26.900000,1.6610,NaN,NaN,2.637004,93.53
3,2011-01-05,27.219999,1.6737,NaN,NaN,2.606665,95.50
4,2011-01-06,27.110001,1.6856,NaN,NaN,2.669766,94.52
...,...,...,...,...,...,...,...
2965,2022-01-26,33.869999,5.4329,NaN,NaN,6.637217,88.74
2966,2022-01-27,33.880001,5.4070,NaN,NaN,6.680007,88.17
2967,2022-01-28,32.540001,5.3676,NaN,NaN,NaN,90.03
2968,2022-01-31,NaN,5.3046,NaN,NaN,6.661627,91.21


# **Tratamento dos dados**
### **Capítulo 3**
<hr>

# **Tratamento dos valores ausentes ou duplicados**
### Capítulo dedicado ao tratamento dos valores ausentes ou duplicados

<hr WIDTH=30% ALIGN=LEFT NOSHADE>

Inicialmente verifica-se a existência de valores ausentes, bem como a quantidade de campos nessa situação
<hr>

In [ ]:
# Verificando dados ausentes (df_final):
s_notanumber = df_final.isna().sum()
if s_notanumber.sum() == 0:
  print('Não foram encontrados campos ausentes em df_final.\n')
else:
  print(f'Foram encontrados campos ausentes em df_final.\nQuantidade de campos ausentes:\n')
  print(s_notanumber)

Foram encontrados campos ausentes em df_final.
Quantidade de campos ausentes:

Date             0
PETR4          222
USD/BRL         79
IPCA          2836
MWh           2837
Fuel_Price     668
Brent          108
dtype: int64


### Como visto, há valores  ausentes em todos os campos (salvo para 'Date' que foi o rótulo de mesclagem)
### Devemos preencher tais valores para a análise dos dados
<hr>

### Como o consumo de energia (MWh) e o IPCA são dados mensais, usaremos a estratégia de preencher os valores ausentes com base no valor aferido para aquele mês, que foi regristrado sempre no primeiro dia. 

### Exemplo:

- O IPCA do mês de Janeiro de 2016 foi de 0.92%, desta forma, todos os dias de 01/2016 receberâo esse valor
<hr>

In [ ]:
# Imputando os valores para as colunas de IPCA e MWh
for col in ['IPCA', 'MWh']:
    df_final[col] = df_final[col].fillna(method='ffill')
# Retornando o tipo de dados da coluna consumo para inteiro
df_final['MWh']=df_final['MWh'].astype(int)

In [ ]:
df_final

,Date,PETR4,USD/BRL,IPCA,MWh,Fuel_Price,Brent
0,2011-01-01,NaN,NaN,0.43,35873744,NaN,NaN
1,2011-01-03,27.000000,1.6470,0.43,35873744,2.584085,94.84
2,2011-01-04,26.900000,1.6610,0.43,35873744,2.637004,93.53
3,2011-01-05,27.219999,1.6737,0.43,35873744,2.606665,95.50
4,2011-01-06,27.110001,1.6856,0.43,35873744,2.669766,94.52
...,...,...,...,...,...,...,...
2965,2022-01-26,33.869999,5.4329,1.15,42486943,6.637217,88.74
2966,2022-01-27,33.880001,5.4070,1.15,42486943,6.680007,88.17
2967,2022-01-28,32.540001,5.3676,1.15,42486943,NaN,90.03
2968,2022-01-31,NaN,5.3046,1.15,42486943,6.661627,91.21


In [ ]:
# Verificando novamente os valores ausentes
df_final.isna().sum()

Date            0
PETR4         222
USD/BRL        79
IPCA            0
MWh             0
Fuel_Price    668
Brent         108
dtype: int64

### Já sabemos que não temos valores medidos de consumo de energia em fevereiro de 2022, em razão do arquivo importado. Assim, vamos limitar o dataframe para datas até 31/01/2022
<hr>

In [ ]:
# Limitando o dataframe de trabalho para data até 31/01/2022
df_final = df_final.loc[df_final.Date < '2022-02-01']
# Resultado
print(f'Foram obtidas {df_final.shape[0]} linhas de dados no dataframe final \n'
      f'limitando as datas até 31 de janeiro de 2022.')

Foram obtidas 2969 linhas de dados no dataframe final 
limitando as datas até 31 de janeiro de 2022.


In [ ]:
# Verificando novamente os valores ausentes
df_final.isna().sum()

Date            0
PETR4         221
USD/BRL        78
IPCA            0
MWh             0
Fuel_Price    667
Brent         107
dtype: int64

### Para os valores ausentes restantes, entende-se que a melhor estratégia seria a remoção, já que a interpolação de uma grande quantidade de campos poderia *mascarar* os resultados
### Utiliza-se o método *dropna()*, com substituição *inplace*

<hr WIDTH=30% ALIGN=LEFT NOSHADE>

In [ ]:
# Removendo dados ausentes
df_final.dropna(inplace=True)

# Reinicializando o index
df_final.reset_index(drop=True, inplace=True)

# Resultado
df_final.head()

,Date,PETR4,USD/BRL,IPCA,MWh,Fuel_Price,Brent
0,2011-01-03,27.000000,1.6470,0.43,35873744,2.584085,94.84
1,2011-01-04,26.900000,1.6610,0.43,35873744,2.637004,93.53
2,2011-01-05,27.219999,1.6737,0.43,35873744,2.606665,95.50
3,2011-01-06,27.110001,1.6856,0.43,35873744,2.669766,94.52
4,2011-01-10,26.980000,1.6888,0.43,35873744,2.614946,95.70


In [ ]:
df_final

,Date,PETR4,USD/BRL,IPCA,MWh,Fuel_Price,Brent
0,2011-01-03,27.000000,1.6470,0.43,35873744,2.584085,94.84
1,2011-01-04,26.900000,1.6610,0.43,35873744,2.637004,93.53
2,2011-01-05,27.219999,1.6737,0.43,35873744,2.606665,95.50
3,2011-01-06,27.110001,1.6856,0.43,35873744,2.669766,94.52
4,2011-01-10,26.980000,1.6888,0.43,35873744,2.614946,95.70
...,...,...,...,...,...,...,...
2171,2022-01-21,31.770000,5.4587,1.15,42486943,6.534684,87.89
2172,2022-01-24,31.950001,5.4882,1.15,42486943,6.672293,86.27
2173,2022-01-25,32.990002,5.4442,1.15,42486943,6.658731,88.20
2174,2022-01-26,33.869999,5.4329,1.15,42486943,6.637217,88.74


In [ ]:
# Verificando novamente os valores ausentes
df_final.isna().sum()

Date          0
PETR4         0
USD/BRL       0
IPCA          0
MWh           0
Fuel_Price    0
Brent         0
dtype: int64

In [ ]:
# Verificando dados nulos (df_final):
s_null = df_final.isnull().sum()
if s_null.sum() == 0:
  print('Não foram encontrados campos nulos em df_final.\n')
else:
  print(f'Foram encontrados campos nulos em df_final.\nQuantidade de campos nulos:\n')
  print(s_null)

Não foram encontrados campos nulos em df_final.



In [ ]:
# Verificando dados duplicados (df_final):
dados_duplicados = df_final.duplicated().sum()
print(f'Total de linhas duplicadas encontradas em df_final: {dados_duplicados} linhas.\n')
# Caso haja dados duplicados, serão já descartados:
if dados_duplicados == 0:
  print('Não há dados duplicados no dataframe de trabalho (df_final).\n')
else:
  df_final.drop_duplicates(inplace=True,  ignore_index=True)
  print(f'Um total de {dados_duplicados} linhas duplicadas foram excluídas do dataframe de trabalho (df_final).\n')
  
print (f'A base de dados após o tratamento dos dados duplicados,\n'
       f'nulos ou ausentes tem {df_final.shape[0]} linhas e {df_final.shape[1]} colunas.\n')

Total de linhas duplicadas encontradas em df_final: 0 linhas.

Não há dados duplicados no dataframe de trabalho (df_final).

A base de dados após o tratamento dos dados duplicados,
nulos ou ausentes tem 2176 linhas e 7 colunas.



In [ ]:
# Salvando o dataset de trabalho df_final
# BackUp
# arq_exportado = r'/content/drive/MyDrive/NOVO_TCC/DADOS/BACKUP_DF/df_final.csv'
arq_exportado = r'C:\Users\Henrique\Documents\TCC-PUCMG\DADOS\BACKUP_DF\df_final.csv'
df_final.to_csv(arq_exportado, index=False)